# Data Workflow

### Python/SQL Setup

In [2]:
from sqlalchemy import create_engine, inspect
from sqlalchemy import text 
import psycopg2
import psycopg2.extras
import json
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import geoalchemy2
from shapely import wkt
from shapely.geometry import Point, Polygon, MultiPolygon

# Helper methods for connecting to and querying postgres server
def pgconnect(credential_filepath, db_schema="public"):
    with open(credential_filepath) as f:
        db_conn_dict = json.load(f)
        host       = db_conn_dict['host']
        db_user    = db_conn_dict['user']
        db_pw      = db_conn_dict['password']
        default_db = db_conn_dict['user']
        try:
            db = create_engine('postgresql+psycopg2://'+db_user+':'+db_pw+'@'+host+'/'+default_db, echo=False)
            conn = db.connect()
            print('Connected successfully.')
        except Exception as e:
            print("Unable to connect to the database.")
            print(e)
            db, conn = None, None
        
        # Making schema selection easier 
        if db_schema != "public" and db != None and conn != None:
            try:
                # IF THIS DOESNT WORK FOR CHRIS OR LUKA, REMOVE THE TEXT() WRAPPER
                conn.execute(text("set search_path to " + db_schema))
                print("Search path set to " + db_schema)
            except Exception as e:
                print("Unable to set search path to "+ db_schema)
                print(e)
                pgdisconnect(db, conn)
                
        return db,conn

def query(conn, sqlcmd, args=None, df=True):
    # conn = database connection object, sqlcmd = string holding command
    result = pd.DataFrame() if df else None
    sqlcmd = text(sqlcmd)
    try:
        if df:
            result = pd.read_sql_query(sqlcmd, conn, params=args)
        else:
            result = conn.execute(sqlcmd, args).fetchall()
            result = result[0] if len(result) == 1 else result
    except Exception as e:
        print("Error encountered: ", e, sep='\n')
    return result

''' in case the text() causes issues down the line
def query(conn, sqlcmd, args=None, df=True):
    result = pd.DataFrame() if df else None
    try:
        if df:
            result = pd.read_sql_query(sqlcmd, conn, params=args)
        else:
            result = conn.execute(sqlcmd, args).fetchall()
            result = result[0] if len(result) == 1 else result
    except Exception as e:
        print("Error encountered: ", e, sep='\n')
    return result
'''
def create_wkt_element(geom, srid):
    if geom.geom_type == 'Polygon':
        geom = MultiPolygon([geom])
    return geoalchemy2.WKTElement(geom.wkt, srid)

# Disconnects from server, able to commit changes here
def pgdisconnect(db, conn, commit=False):
    try:
        if commit:
            conn.commit()
            print("Changes committed.")
        conn.close()
        db.dispose()
        print("Disconnected from database.")
        return True
    except Exception as E:
        print("Unable to disconnect from the database.")
        print(e)
        return False

# Utilises disconnect + reconnect methods, best for unique servers like Yash's where you must commit or no change is saved
def pgreconnect(credentials, old_db, old_conn, db_schema="public", commit=False):
    if pgdisconnect(old_db, old_conn, commit):
        return pgconnect(credentials, db_schema)
    else:
        print("No new connection made.")
        return None, None

In [3]:
credentials = "Credentials.json"
db, conn = pgconnect(credentials)

Connected successfully.


In [4]:
# Creating new schema
sql = """
CREATE SCHEMA IF NOT EXISTS sa2;
SET search_path TO sa2;
"""

conn.execute(text(sql))

In [4]:
# Adding PostGIS to sa2 schema
sql = """
CREATE EXTENSION IF NOT EXISTS postgis;

UPDATE pg_extension
SET extrelocatable = TRUE
WHERE extname = 'postgis';

ALTER EXTENSION postgis
SET SCHEMA sa2;
"""

conn.execute(text(sql))
query(conn, "SELECT PostGIS_version()") 

,postgis_version
0,3.3 USE_GEOS=1 USE_PROJ=1 USE_STATS=1


In [5]:
#db, conn = pgreconnect(credentials, db, conn, 'sa2', False)
#query(conn, "SET SEARCH_PATH TO sa2")

### Task 1: Cleaning & Importing 

#### SA2 Regions dataset


In [6]:
# Loading and inspecting the dataset
path = "SA2_2021_AUST_SHP_GDA2020/SA2_2021_AUST_GDA2020.shp"
regions = gpd.read_file(path)
print(regions.shape)
print(regions.columns)
print(regions.dtypes)
regions.head()

(2473, 17)
Index(['SA2_CODE21', 'SA2_NAME21', 'CHG_FLAG21', 'CHG_LBL21', 'SA3_CODE21',
       'SA3_NAME21', 'SA4_CODE21', 'SA4_NAME21', 'GCC_CODE21', 'GCC_NAME21',
       'STE_CODE21', 'STE_NAME21', 'AUS_CODE21', 'AUS_NAME21', 'AREASQKM21',
       'LOCI_URI21', 'geometry'],
      dtype='object')
SA2_CODE21      object
SA2_NAME21      object
CHG_FLAG21      object
CHG_LBL21       object
SA3_CODE21      object
SA3_NAME21      object
SA4_CODE21      object
SA4_NAME21      object
GCC_CODE21      object
GCC_NAME21      object
STE_CODE21      object
STE_NAME21      object
AUS_CODE21      object
AUS_NAME21      object
AREASQKM21     float64
LOCI_URI21      object
geometry      geometry
dtype: object


,SA2_CODE21,SA2_NAME21,CHG_FLAG21,CHG_LBL21,SA3_CODE21,SA3_NAME21,SA4_CODE21,SA4_NAME21,GCC_CODE21,GCC_NAME21,STE_CODE21,STE_NAME21,AUS_CODE21,AUS_NAME21,AREASQKM21,LOCI_URI21,geometry
0,101021007,Braidwood,0,No change,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,3418.3525,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((149.58424 -35.44426, 149.58444 -35.4..."
1,101021008,Karabar,0,No change,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,6.9825,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((149.21899 -35.36738, 149.21800 -35.3..."
2,101021009,Queanbeyan,0,No change,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,4.7620,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((149.21326 -35.34325, 149.21619 -35.3..."
3,101021010,Queanbeyan - East,0,No change,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,13.0032,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((149.24034 -35.34781, 149.24024 -35.3..."
4,101021012,Queanbeyan West - Jerrabomberra,0,No change,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,13.6748,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((149.19572 -35.36126, 149.19970 -35.3..."


In [7]:
# Filter for only the SA2 regions in greater Sydney
regions = regions[regions["GCC_CODE21"] == "1GSYD"]

In [8]:
# Removing unnecessary columns
to_remove = ["CHG_FLAG21", "CHG_LBL21", "GCC_CODE21", "GCC_NAME21", "STE_CODE21", "STE_NAME21", "AUS_CODE21", "AUS_NAME21", "LOCI_URI21"]
regions.drop(to_remove, axis=1, inplace=True)

In [9]:
# Converting all POLYGON types to MULTIPOLYGON
regions["geom"] = regions["geometry"].apply(lambda x: create_wkt_element(geom=x,srid=7844))
regions.drop(columns="geometry", inplace=True)

In [10]:
# Renaming columns
regions.rename({
    "SA2_CODE21":"sa2_code",
    "SA2_NAME21":"sa2_name",
    "SA3_CODE21":"sa3_code",
    "SA3_NAME21":"sa3_name",
    "SA4_CODE21":"sa4_code",
    "SA4_NAME21":"sa4_name",
    "AREASQKM21":"area_sqkm",
    "geom":"geometry"
}, axis=1, inplace=True)

In [11]:
# Inspecting the dataset after modifcation
regions.head()

,sa2_code,sa2_name,sa3_code,sa3_name,sa4_code,sa4_name,area_sqkm,geometry
28,102011028,Avoca Beach - Copacabana,10201,Gosford,102,Central Coast,6.4376,MULTIPOLYGON (((151.413733024921 -33.465580583...
29,102011029,Box Head - MacMasters Beach,10201,Gosford,102,Central Coast,32.0802,MULTIPOLYGON (((151.37484081570685 -33.5005199...
30,102011030,Calga - Kulnura,10201,Gosford,102,Central Coast,767.9512,MULTIPOLYGON (((151.20449037540152 -33.5328022...
31,102011031,Erina - Green Point,10201,Gosford,102,Central Coast,33.7934,MULTIPOLYGON (((151.37193611462118 -33.4369790...
32,102011032,Gosford - Springfield,10201,Gosford,102,Central Coast,16.9123,MULTIPOLYGON (((151.32348639265098 -33.4277852...


In [12]:
# Correcting datatypes
regions = regions.astype({"sa2_code":"int64", "sa3_code":"int64", "sa4_code":"int64"})

# Adding to postgresql database
sql = """
DROP TABLE IF EXISTS regions CASCADE;
CREATE TABLE regions (
    sa2_code INTEGER PRIMARY KEY,
    sa2_name VARCHAR(100) UNIQUE NOT NULL,
    sa3_code INTEGER NOT NULL,
    sa3_name VARCHAR(100) NOT NULL,
    sa4_code INTEGER NOT NULL,
    sa4_name VARCHAR(100) NOT NULL,
    area_sqkm NUMERIC NOT NULL,
    geometry GEOMETRY(MULTIPOLYGON, 7844) NOT NULL
);"""
conn.execute(text(sql));

regions.to_sql(name="regions", con=conn, schema="sa2", if_exists="append", index=False,
              dtype={'geometry': geoalchemy2.Geometry('MULTIPOLYGON', 7844)})

query(conn, "SELECT * FROM regions LIMIT 5;")

,sa2_code,sa2_name,sa3_code,sa3_name,sa4_code,sa4_name,area_sqkm,geometry
0,102011028,Avoca Beach - Copacabana,10201,Gosford,102,Central Coast,6.4376,0106000020A41E0000010000000103000000010000005E...
1,102011029,Box Head - MacMasters Beach,10201,Gosford,102,Central Coast,32.0802,0106000020A41E00000100000001030000000100000010...
2,102011030,Calga - Kulnura,10201,Gosford,102,Central Coast,767.9512,0106000020A41E00000200000001030000000100000085...
3,102011031,Erina - Green Point,10201,Gosford,102,Central Coast,33.7934,0106000020A41E00000100000001030000000100000041...
4,102011032,Gosford - Springfield,10201,Gosford,102,Central Coast,16.9123,0106000020A41E0000010000000103000000010000007E...


#### Businesses dataset

In [13]:
# Loading and inspecting the dataset
business = pd.read_csv("Businesses.csv")
print(business.shape)
print(business.columns)
print(business.dtypes)

(12217, 11)
Index(['industry_code', 'industry_name', 'sa2_code', 'sa2_name',
       '0_to_50k_businesses', '50k_to_200k_businesses',
       '200k_to_2m_businesses', '2m_to_5m_businesses', '5m_to_10m_businesses',
       '10m_or_more_businesses', 'total_businesses'],
      dtype='object')
industry_code             object
industry_name             object
sa2_code                   int64
sa2_name                  object
0_to_50k_businesses        int64
50k_to_200k_businesses     int64
200k_to_2m_businesses      int64
2m_to_5m_businesses        int64
5m_to_10m_businesses       int64
10m_or_more_businesses     int64
total_businesses           int64
dtype: object


In [14]:
# View of the particular industries accounted for in each sa2 region
business[["industry_code", "industry_name"]].drop_duplicates()

,industry_code,industry_name
0,A,"Agriculture, Forestry and Fishing"
643,B,Mining
1286,C,Manufacturing
1929,D,"Electricity, Gas, Water and Waste Services"
2572,E,Construction
3215,F,Wholesale Trade
3858,G,Retail Trade
4501,H,Accommodation and Food Services
5144,I,"Transport, Postal and Warehousing"
5787,J,Information Media and Telecommunications


To be able to correctly join this table with the regions dataset and improve the accuracy of our analysis, we will need to remove the SA2 code for "currently unknown" (199999499) from the table:

In [15]:
business = business[business["sa2_code"] != 199999499]

In addition, we will also filter for only the SA2 regions in greater Sydney:

In [16]:
business = business.loc[business["sa2_code"].isin(regions["sa2_code"])]

In addition, we should also check that the count of the businesses in each size category adds to the total_businesses column:

In [17]:
# Checking if sum of businesses equals total businesses column
accounted_businesses = sum(business.loc[:,"0_to_50k_businesses":"5m_to_10m_businesses":1].sum(axis=1) == business["total_businesses"])
prop_correct_business_sum = accounted_businesses/business.shape[0]
round(prop_correct_business_sum, 2)

0.33

To improve the quality of the data analysis, these total_businesses values will be corrected to follow the sum of the number of businesses in each category:

In [18]:
business["total_businesses"] = business.loc[:,"0_to_50k_businesses":"5m_to_10m_businesses":1].sum(axis=1)

Finally, we need to export this DataFrame from pandas into the postgresql database:

In [19]:
# Selecting required columns
business = business[["sa2_code", "industry_code", "industry_name", "total_businesses"]]

# Adding to postgresql database
sql = """
DROP TABLE IF EXISTS business;
CREATE TABLE business (
    sa2_code INTEGER NOT NULL REFERENCES regions (sa2_code),
    industry_code CHAR(1) NOT NULL,
    industry_name VARCHAR(50) NOT NULL,
    total_businesses INTEGER NOT NULL,
    
    PRIMARY KEY (sa2_code, industry_code)
);"""
conn.execute(text(sql));

business.to_sql("business", conn, schema="sa2", if_exists="append", index=False)

query(conn, "SELECT * FROM business LIMIT 5;")

,sa2_code,industry_code,industry_name,total_businesses
0,102011028,A,"Agriculture, Forestry and Fishing",6
1,102011029,A,"Agriculture, Forestry and Fishing",17
2,102011030,A,"Agriculture, Forestry and Fishing",214
3,102011031,A,"Agriculture, Forestry and Fishing",27
4,102011032,A,"Agriculture, Forestry and Fishing",19


#### Stops dataset
Since the stops file provided is in a GTFS format, we will first load it in as a csv file with pandas, then convert it into a GeoDataFrame with geopandas.

Useful link: https://developers.google.com/transit/gtfs/reference#stopstxt

In [20]:
## Loading and inspecting DataFrame
stops_df = pd.read_csv("Stops.txt")
print(stops_df.shape)
print(stops_df.columns)

# Filtering for only the stops
stops_df = stops_df[stops_df["location_type"].isna() & stops_df["stop_code"].notna()]

# Correcting datatypes
stops_df[["wheelchair_boarding"]] = stops_df[["wheelchair_boarding"]].replace({0:np.nan, 1:True, 2:False})
stops_df = stops_df.astype({"stop_code":"int64", "wheelchair_boarding":bool})

print(stops_df.dtypes)

(114718, 9)
Index(['stop_id', 'stop_code', 'stop_name', 'stop_lat', 'stop_lon',
       'location_type', 'parent_station', 'wheelchair_boarding',
       'platform_code'],
      dtype='object')
stop_id                 object
stop_code                int64
stop_name               object
stop_lat               float64
stop_lon               float64
location_type          float64
parent_station          object
wheelchair_boarding       bool
platform_code           object
dtype: object


Now we need to convert the DataFrame into a GeoDataFrame with geopandas:

In [21]:
# Converting into GeoDataFrame
stops_gdf = gpd.GeoDataFrame(stops_df, geometry=gpd.points_from_xy(stops_df.stop_lon, stops_df.stop_lat)).set_crs(epsg=4326)


# Removing unnecessary columns
stops_gdf.drop(["stop_id", "stop_lon", "parent_station", "stop_lat", "location_type", "platform_code"], axis=1, inplace=True)
stops_gdf.head()

# Conducting WKT conversion
stops_gdf['geometry'] = stops_gdf['geometry'].apply(lambda x: geoalchemy2.WKTElement(x.wkt, 4326))

stops_gdf.head()

/home/chris/anaconda3/lib/python3.9/site-packages/geopandas/geodataframe.py:1442: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


,stop_code,stop_name,wheelchair_boarding,geometry
0,200039,"Central Station, Eddy Av, Stand A",True,POINT (151.20666465471 -33.8822064874687)
1,200054,"Central Station, Eddy Av, Stand D",True,POINT (151.20699145565 -33.8820421431408)
4,201646,"Redfern Station, Gibbons St, Stand B",True,POINT (151.198881722942 -33.8933293130144)
5,204230,"St Peters Station, King St",True,POINT (151.181117167927 -33.9063142029908)
6,204311,King St Opp St Peters Station,True,POINT (151.181371008764 -33.9064227004899)


Finally, we need to export this GeoDataFrame from geopandas into the postgresql database:

In [22]:
# Adding to postgresql database
sql = """
DROP TABLE IF EXISTS stops;
CREATE TABLE stops (
    stop_code INTEGER PRIMARY KEY,
    stop_name VARCHAR(100) NOT NULL,
    wheelchair_boarding BOOLEAN,
    geometry GEOMETRY(POINT, 4326) NOT NULL
);"""
conn.execute(text(sql));

stops_gdf.to_sql("stops", conn, schema="sa2", if_exists="append", index=False, dtype={'geometry': geoalchemy2.Geometry('POINT', 4326)})

query(conn, "SELECT * FROM stops LIMIT 5;")

,stop_code,stop_name,wheelchair_boarding,geometry
0,200039,"Central Station, Eddy Av, Stand A",True,0101000020E6100000FFA631FF9CE66240A1FF6524ECF0...
1,200054,"Central Station, Eddy Av, Stand D",True,0101000020E61000002F928BAC9FE66240E33DC7C1E6F0...
2,201646,"Redfern Station, Gibbons St, Stand B",True,0101000020E6100000DBF9333D5DE662403DFA6B9D58F2...
3,204230,"St Peters Station, King St",True,0101000020E6100000B31F3BB6CBE5624076DF921A02F4...
4,204311,King St Opp St Peters Station,True,0101000020E6100000AFE292CACDE56240627AB7A805F4...


#### Polls dataset
Since the polls dataset already has a sql POINT object already implemented in the table, we will set the column to be the geometry of the GeoDataFrame. Since for the purposes of this assignment we need to assign each polling place a location, we will only use the entries where this column is not NULL.

In [23]:
## Loading and inspecting DataFrame
polls_df = pd.read_csv("PollingPlaces2019.csv")
print(polls_df.shape)
print(polls_df.columns)
print(polls_df.dtypes)

# Updating datatype of the_geom
polls_df = polls_df[polls_df['the_geom'].notna()]
polls_df["the_geom"] = polls_df["the_geom"].apply(wkt.loads)

(2930, 17)
Index(['FID', 'state', 'division_id', 'division_name', 'polling_place_id',
       'polling_place_type_id', 'polling_place_name', 'premises_name',
       'premises_address_1', 'premises_address_2', 'premises_address_3',
       'premises_suburb', 'premises_state_abbreviation', 'premises_post_code',
       'latitude', 'longitude', 'the_geom'],
      dtype='object')
FID                             object
state                           object
division_id                      int64
division_name                   object
polling_place_id                 int64
polling_place_type_id            int64
polling_place_name              object
premises_name                   object
premises_address_1              object
premises_address_2              object
premises_address_3              object
premises_suburb                 object
premises_state_abbreviation     object
premises_post_code             float64
latitude                       float64
longitude                      float64


Now we need to convert the DataFrame into a GeoDataFrame with geopandas:

In [24]:
# Loading into GeoDataFrame
polls_gdf = gpd.GeoDataFrame(polls_df, geometry=gpd.points_from_xy(polls_df.longitude, polls_df.latitude)).set_crs(epsg=4283)

# Removing unnecessary columns
polls_gdf.drop(["longitude", "latitude", "the_geom", "FID", "state", "polling_place_name", "polling_place_type_id", "premises_name", "premises_address_1", "premises_address_2", "premises_address_3", "premises_suburb", "premises_state_abbreviation", "premises_post_code"], axis=1, inplace=True)

# Conducting WKT conversion
polls_gdf['geometry'] = polls_gdf['geometry'].apply(lambda x: geoalchemy2.WKTElement(x.wkt, 4283))

polls_gdf.head()

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/geopandas/geodataframe.py:1537: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


,division_id,division_name,polling_place_id,geometry
13,103,Banks,58,POINT (151.081 -33.9847)
15,111,Chifley,392,POINT (150.817 -33.7475)
16,103,Banks,31,POINT (151.1148974 -33.9767897)
17,103,Banks,67,POINT (151.111 -33.9756)
18,103,Banks,56500,POINT (151.075 -33.9413)


Finally, we need to export this GeoDataFrame from geopandas into the postgresql database:

In [10]:
# Adding to postgresql database
sql = """
DROP TABLE IF EXISTS polls;
CREATE TABLE polls (
    polling_place_id INTEGER PRIMARY KEY,
    division_id INTEGER NOT NULL,
    division_name VARCHAR(50) NOT NULL,
    geometry GEOMETRY(POINT, 4283) NOT NULL
);"""

conn.execute(text(sql));

polls_gdf.to_sql("polls", conn, schema="sa2", if_exists="append", index=False, dtype={'geometry': geoalchemy2.Geometry('POINT', 4283)})

query(conn, "SELECT * FROM polls LIMIT 5")

,polling_place_id,division_id,division_name,geometry
0,58,103,Banks,0101000020BB1000003BDF4F8D97E26240832F4CA60AFE...
1,392,111,Chifley,0101000020BB100000A01A2FDD24DA624048E17A14AEDF...
2,31,103,Banks,0101000020BB100000A7EC4F3DADE36240EA48E47107FD...
3,67,103,Banks,0101000020BB100000643BDF4F8DE3624022FDF675E0FC...
4,56500,103,Banks,0101000020BB1000006666666666E26240C6DCB5847CF8...


#### Schools dataset

In [26]:
# Loading and inspecting the dataset
path = "Catchments/catchments_primary.shp"
primary = gpd.read_file(path)
print(primary.shape)
print(primary.columns)
print(primary.dtypes)
primary.head()

(1662, 19)
Index(['USE_ID', 'CATCH_TYPE', 'USE_DESC', 'ADD_DATE', 'KINDERGART', 'YEAR1',
       'YEAR2', 'YEAR3', 'YEAR4', 'YEAR5', 'YEAR6', 'YEAR7', 'YEAR8', 'YEAR9',
       'YEAR10', 'YEAR11', 'YEAR12', 'PRIORITY', 'geometry'],
      dtype='object')
USE_ID          object
CATCH_TYPE      object
USE_DESC        object
ADD_DATE        object
KINDERGART      object
YEAR1           object
YEAR2           object
YEAR3           object
YEAR4           object
YEAR5           object
YEAR6           object
YEAR7           object
YEAR8           object
YEAR9           object
YEAR10          object
YEAR11          object
YEAR12          object
PRIORITY        object
geometry      geometry
dtype: object


,USE_ID,CATCH_TYPE,USE_DESC,ADD_DATE,KINDERGART,YEAR1,YEAR2,YEAR3,YEAR4,YEAR5,YEAR6,YEAR7,YEAR8,YEAR9,YEAR10,YEAR11,YEAR12,PRIORITY,geometry
0,2838,PRIMARY,Parklea PS,20181210,Y,Y,Y,Y,Y,Y,Y,N,N,N,N,N,N,NaN,"POLYGON ((150.93564 -33.71612, 150.93715 -33.7..."
1,2404,PRIMARY,Lindfield EPS,20211219,Y,Y,Y,Y,Y,Y,Y,N,N,N,N,N,N,NaN,"POLYGON ((151.18336 -33.74748, 151.18443 -33.7..."
2,4393,PRIMARY,Carlingford WPS,20220223,Y,Y,Y,Y,Y,Y,Y,N,N,N,N,N,N,NaN,"POLYGON ((151.04518 -33.77303, 151.04526 -33.7..."
3,4615,PRIMARY,Caddies Ck PS,20181210,Y,Y,Y,Y,Y,Y,Y,N,N,N,N,N,N,NaN,"POLYGON ((150.92567 -33.72960, 150.92602 -33.7..."
4,3918,PRIMARY,Killara PS,20211219,Y,Y,Y,Y,Y,Y,Y,N,N,N,N,N,N,NaN,"POLYGON ((151.15379 -33.75586, 151.15404 -33.7..."


In [27]:
# Loading and inspecting the dataset
path = "Catchments/catchments_secondary.shp"
secondary = gpd.read_file(path)
print(secondary.shape)
print(secondary.columns)
print(secondary.dtypes)
secondary.head()

(436, 19)
Index(['USE_ID', 'CATCH_TYPE', 'USE_DESC', 'ADD_DATE', 'KINDERGART', 'YEAR1',
       'YEAR2', 'YEAR3', 'YEAR4', 'YEAR5', 'YEAR6', 'YEAR7', 'YEAR8', 'YEAR9',
       'YEAR10', 'YEAR11', 'YEAR12', 'PRIORITY', 'geometry'],
      dtype='object')
USE_ID          object
CATCH_TYPE      object
USE_DESC        object
ADD_DATE        object
KINDERGART      object
YEAR1           object
YEAR2           object
YEAR3           object
YEAR4           object
YEAR5           object
YEAR6           object
YEAR7           object
YEAR8           object
YEAR9           object
YEAR10          object
YEAR11          object
YEAR12          object
PRIORITY        object
geometry      geometry
dtype: object


,USE_ID,CATCH_TYPE,USE_DESC,ADD_DATE,KINDERGART,YEAR1,YEAR2,YEAR3,YEAR4,YEAR5,YEAR6,YEAR7,YEAR8,YEAR9,YEAR10,YEAR11,YEAR12,PRIORITY,geometry
0,8503,HIGH_COED,Billabong HS,20200507,N,N,N,N,N,N,N,Y,Y,Y,Y,Y,Y,NaN,"POLYGON ((146.67182 -35.31444, 146.68930 -35.3..."
1,8266,HIGH_COED,James Fallon HS,20200507,N,N,N,N,N,N,N,Y,Y,Y,Y,Y,Y,NaN,"POLYGON ((147.08734 -35.86271, 147.10413 -35.8..."
2,8505,HIGH_COED,Murray HS,20200507,N,N,N,N,N,N,N,Y,Y,Y,Y,Y,Y,NaN,"POLYGON ((146.81448 -35.78341, 146.81250 -35.7..."
3,8458,HIGH_COED,Kingswood HS,20201016,N,N,N,N,N,N,N,Y,Y,Y,Y,Y,Y,NaN,"MULTIPOLYGON (((150.68600 -33.74031, 150.68631..."
4,8559,HIGH_COED,Jamison HS,20201016,N,N,N,N,N,N,N,Y,Y,Y,Y,Y,Y,NaN,"POLYGON ((150.69513 -33.75627, 150.68936 -33.7..."


In [28]:
# Loading and inspecting the dataset
path = "Catchments/catchments_future.shp"
future = gpd.read_file(path)
print(future.shape)
print(future.columns)
print(future.dtypes)
future.head()

(30, 18)
Index(['USE_ID', 'CATCH_TYPE', 'USE_DESC', 'ADD_DATE', 'KINDERGART', 'YEAR1',
       'YEAR2', 'YEAR3', 'YEAR4', 'YEAR5', 'YEAR6', 'YEAR7', 'YEAR8', 'YEAR9',
       'YEAR10', 'YEAR11', 'YEAR12', 'geometry'],
      dtype='object')
USE_ID          object
CATCH_TYPE      object
USE_DESC        object
ADD_DATE        object
KINDERGART       int64
YEAR1            int64
YEAR2            int64
YEAR3            int64
YEAR4            int64
YEAR5            int64
YEAR6            int64
YEAR7            int64
YEAR8            int64
YEAR9            int64
YEAR10           int64
YEAR11           int64
YEAR12           int64
geometry      geometry
dtype: object


,USE_ID,CATCH_TYPE,USE_DESC,ADD_DATE,KINDERGART,YEAR1,YEAR2,YEAR3,YEAR4,YEAR5,YEAR6,YEAR7,YEAR8,YEAR9,YEAR10,YEAR11,YEAR12,geometry
0,8416,HIGH_COED,Ku-ring-gai HS,20230114,0,0,0,0,0,0,0,2024,2024,2024,2024,2024,2024,"POLYGON ((151.19849 -33.53990, 151.19945 -33.5..."
1,8161,HIGH_BOYS,Randwick BHS,20200220,0,0,0,0,0,0,0,2024,2024,2024,2024,2024,2024,"POLYGON ((151.27152 -33.91402, 151.27152 -33.9..."
2,8539,HIGH_COED,SSC Blackwattle Bay,20220609,0,0,0,0,0,0,0,0,0,0,0,2024,2024,"POLYGON ((151.15292 -33.83939, 151.16144 -33.8..."
3,8400,HIGH_COED,St Ives HS,20230114,0,0,0,0,0,0,0,2024,2024,2024,2024,2024,2024,"POLYGON ((151.17794 -33.69820, 151.17859 -33.6..."
4,8555,HIGH_COED,Rose Bay SC,20200220,0,0,0,0,0,0,0,2024,2024,2024,2024,2024,2024,"POLYGON ((151.28072 -33.83287, 151.28095 -33.8..."


Since the future catchments datset 

In [29]:
#Joining the datasets
catchments = pd.concat([primary, secondary, future])
catchments.drop_duplicates(["USE_ID", "CATCH_TYPE"], keep="last", inplace=True)

#Removing Columns
catchments = catchments.drop(columns = ["KINDERGART", "YEAR1", "YEAR2", "YEAR3", "YEAR4", "YEAR5", "YEAR6", "YEAR7", "YEAR8", "YEAR9", "YEAR10", "YEAR11", "YEAR12", "PRIORITY"])

# Converting all POLYGON types to MULTIPOLYGON
catchments["geom"] = catchments["geometry"].apply(lambda x: create_wkt_element(geom=x,srid=7844))
catchments.drop(columns="geometry", inplace=True)
catchments.rename({"geom":"geometry"}, axis=1, inplace=True)
catchments.columns = catchments.columns.str.lower()

catchments_gdf = gpd.GeoDataFrame(catchments)

Export to postgresql:

In [30]:
sql = """
DROP TABLE IF EXISTS catchments;
CREATE TABLE catchments (
    use_id INTEGER NOT NULL,
    catch_type VARCHAR(100) NOT NULL,
    use_desc VARCHAR(100) NOT NULL,
    add_date DATE,
    geometry GEOMETRY(MULTIPOLYGON, 7844) NOT NULL,
    
    PRIMARY KEY (use_id, catch_type)
);"""
conn.execute(text(sql))

catchments_gdf.to_sql("catchments", conn, schema="sa2", if_exists="append", index=False, dtype={'geometry': geoalchemy2.Geometry('MULTIPOLYGON', 7844)})

query(conn, "SELECT * FROM catchments LIMIT 5;")

,use_id,catch_type,use_desc,add_date,geometry
0,2838,PRIMARY,Parklea PS,2018-12-10,0106000020A41E00000100000001030000000100000078...
1,2404,PRIMARY,Lindfield EPS,2021-12-19,0106000020A41E000001000000010300000001000000BE...
2,4393,PRIMARY,Carlingford WPS,2022-02-23,0106000020A41E00000100000001030000000100000065...
3,1659,PRIMARY,Corowa SPS,2020-04-04,0106000020A41E00000100000001030000000100000007...
4,3867,PRIMARY,Lake Illawarra SPS,2022-06-19,0106000020A41E00000100000001030000000100000022...


#### Population dataset
The population dataset provides us with the population of each SA2 region, which is further paritioned by age. For the project, this table will be used to calculate the required metrics that follow a 'per capita' basis. The dataset provides a unique SA2 code for each row, so we will use this field as the primary key of the dataset, so that it can easily join with the SA2 dataset.

In [31]:
# Loading and inspecting the dataset
population = pd.read_csv("Population.csv")
print(population.shape)
print(population.columns)
print(population.dtypes)

(373, 21)
Index(['sa2_code', 'sa2_name', '0-4_people', '5-9_people', '10-14_people',
       '15-19_people', '20-24_people', '25-29_people', '30-34_people',
       '35-39_people', '40-44_people', '45-49_people', '50-54_people',
       '55-59_people', '60-64_people', '65-69_people', '70-74_people',
       '75-79_people', '80-84_people', '85-and-over_people', 'total_people'],
      dtype='object')
sa2_code               int64
sa2_name              object
0-4_people             int64
5-9_people             int64
10-14_people           int64
15-19_people           int64
20-24_people           int64
25-29_people           int64
30-34_people           int64
35-39_people           int64
40-44_people           int64
45-49_people           int64
50-54_people           int64
55-59_people           int64
60-64_people           int64
65-69_people           int64
70-74_people           int64
75-79_people           int64
80-84_people           int64
85-and-over_people     int64
total_people          

Similarly to the Business dataset, we should verify that the sum of the people in each of the columns equals the total_people column:

In [32]:
accounted_population = sum(population.loc[:,"0-4_people":"85-and-over_people":1].sum(axis=1) == population["total_people"])
prop_correct_population_sum = accounted_population/population.shape[0]
round(prop_correct_population_sum, 2)

1.0

As we can see the total_people column correctly adds up each of the rows of people, allowing us to use the field for our analysis. We will also create the young_people column to calculate the schools metric in task 2: 

In [33]:
# Adding young_people column
population["young_people"] = population.loc[:,"0-4_people":"15-19_people":1].sum(axis=1)

Finally, we need to import the DataFrame into the postgresql database:

In [34]:
# Taking only required rows
population = population[["sa2_code", "young_people", "total_people"]]

# Adding to postgresql database
sql = """
DROP TABLE IF EXISTS population;
CREATE TABLE population (
    sa2_code INTEGER PRIMARY KEY REFERENCES regions (sa2_code),
    young_people INTEGER NOT NULL,
    total_people INTEGER NOT NULL
);"""
conn.execute(text(sql))

population.to_sql("population", conn, schema="sa2", if_exists="append", index=False)

query(conn, "SELECT * FROM population LIMIT 5;")

,sa2_code,young_people,total_people
0,102011028,2121,7530
1,102011029,2471,11052
2,102011030,961,4748
3,102011031,3205,14803
4,102011032,4364,21346


In [53]:
#db, conn = pgreconnect(credentials, db, conn, 'sa2', True)

### Task 2: Computing the "well-resourced" Score

We create a set of views holding different joins, combining these at the end to generate calculations. We can not create one large joined view initially due to the duplication of rows for each matching catchment, stop and polling place.

#### Non spatial joins

We first carry out non-spatial joins, noting that two rows are returned for each sa2 code. We ensure all sa2 regions from the 'regions' dataset are maintained by left joining with non-spatial data, in the process filtering out data which does not corresponding with an sa2 region recorded thus unusable in downstream analysis. 

In [37]:
sql="""
CREATE OR REPLACE VIEW non_spat_join AS
SELECT *
FROM 
    regions r LEFT JOIN population p USING (sa2_code)
                LEFT JOIN business b USING (sa2_code)
WHERE
    b.industry_code = 'G' OR
    b.industry_code = 'Q';
               
SELECT *
FROM non_spat_join
ORDER BY sa2_code
LIMIT 10;
""" 
# This generates 2 rows for each sa2 -> make sure to choose average(population_metric) if just taking popn, or filter by one industry code
query(conn, sql)

,sa2_code,sa2_name,sa3_code,sa3_name,sa4_code,sa4_name,area_sqkm,geometry,young_people,total_people,industry_code,industry_name,total_businesses
0,102011028,Avoca Beach - Copacabana,10201,Gosford,102,Central Coast,6.4376,0106000020A41E0000010000000103000000010000005E...,2121,7530,G,Retail Trade,45
1,102011028,Avoca Beach - Copacabana,10201,Gosford,102,Central Coast,6.4376,0106000020A41E0000010000000103000000010000005E...,2121,7530,Q,Health Care and Social Assistance,74
2,102011029,Box Head - MacMasters Beach,10201,Gosford,102,Central Coast,32.0802,0106000020A41E00000100000001030000000100000010...,2471,11052,G,Retail Trade,51
3,102011029,Box Head - MacMasters Beach,10201,Gosford,102,Central Coast,32.0802,0106000020A41E00000100000001030000000100000010...,2471,11052,Q,Health Care and Social Assistance,59
4,102011030,Calga - Kulnura,10201,Gosford,102,Central Coast,767.9512,0106000020A41E00000200000001030000000100000085...,961,4748,G,Retail Trade,49
5,102011030,Calga - Kulnura,10201,Gosford,102,Central Coast,767.9512,0106000020A41E00000200000001030000000100000085...,961,4748,Q,Health Care and Social Assistance,43
6,102011031,Erina - Green Point,10201,Gosford,102,Central Coast,33.7934,0106000020A41E00000100000001030000000100000041...,3205,14803,G,Retail Trade,144
7,102011031,Erina - Green Point,10201,Gosford,102,Central Coast,33.7934,0106000020A41E00000100000001030000000100000041...,3205,14803,Q,Health Care and Social Assistance,204
8,102011032,Gosford - Springfield,10201,Gosford,102,Central Coast,16.9123,0106000020A41E0000010000000103000000010000007E...,4364,21346,G,Retail Trade,174
9,102011032,Gosford - Springfield,10201,Gosford,102,Central Coast,16.9123,0106000020A41E0000010000000103000000010000007E...,4364,21346,Q,Health Care and Social Assistance,300


We then calculate metrics for non spatial dataset (business info).

In [41]:
sql="""
CREATE OR REPLACE VIEW retail_agg AS
SELECT 
    sa2_code, 
    industry_name,
    total_businesses as "total_retail",
    (total_businesses::float/total_people::float) * 1000 as "retail_per_1000"
FROM 
    non_spat_join
WHERE
    industry_code = 'G';
    
CREATE OR REPLACE VIEW health_agg AS
SELECT
    sa2_code,
    industry_name,
    total_businesses as "total_health",
    (total_businesses::float/total_people::float) * 1000 as "health_per_1000"
FROM 
    non_spat_join
WHERE
    industry_code = 'Q';

SELECT *
FROM retail_agg FULL OUTER JOIN health_agg USING (sa2_code)
LIMIT 10;
""" 

query(conn, sql)

,sa2_code,industry_name,total_retail,retail_per_1000,industry_name,total_health,health_per_1000
0,102011028,Retail Trade,45,5.976096,Health Care and Social Assistance,74,9.827357
1,102011029,Retail Trade,51,4.614549,Health Care and Social Assistance,59,5.338400
2,102011030,Retail Trade,49,10.320135,Health Care and Social Assistance,43,9.056445
3,102011031,Retail Trade,144,9.727758,Health Care and Social Assistance,204,13.780990
4,102011032,Retail Trade,174,8.151410,Health Care and Social Assistance,300,14.054155
5,102011033,Retail Trade,24,3.682111,Health Care and Social Assistance,21,3.221847
6,102011034,Retail Trade,39,5.112743,Health Care and Social Assistance,56,7.341374
7,102011035,Retail Trade,16,2.225003,Health Care and Social Assistance,24,3.337505
8,102011036,Retail Trade,29,3.520699,Health Care and Social Assistance,25,3.035086
9,102011037,Retail Trade,22,3.346008,Health Care and Social Assistance,36,5.475285


#### Spatial Joins

Now we conduct spatial joins, and calculate metrics. For catchments we join if intersection occurs, at least one point is shared between the two geometries, allowing us to capture which sa2 regions have access to a catchment for a school. For both stops and polls spatial data, we have geometries as points and thus we join on an sa2 region with the contains spatial relationship.

In [54]:
# need to take average of populations, as multiple rows with same popn found for each intersecting catchment
# and to ensure counts arent interfered with we take only one industry code 
# if no young people in a sa2 code, THEN -1 RETURNED FOR NOW
sql="""
CREATE OR REPLACE VIEW catchment_agg AS
SELECT 
    r.sa2_code,
    count(c.use_desc) as "total_schools",
    CASE
        WHEN avg(young_people) = 0 THEN -1
        ELSE count(c.use_desc)/avg(young_people) * 1000 
    END as "schools_per_1000_young"
FROM 
    regions r JOIN catchments c ON ST_INTERSECTS(r.geometry, ST_TRANSFORM(c.geometry, 7844))
                JOIN non_spat_join USING (sa2_code)
WHERE
    industry_code = 'G'
GROUP BY
    r.sa2_code;
    
CREATE OR REPLACE VIEW stops_agg AS
SELECT 
    r.sa2_code,
    count(s.stop_name) as "total_stops"
FROM 
    regions r JOIN stops s ON ST_CONTAINS(r.geometry, ST_TRANSFORM(s.geometry, 7844))
GROUP BY
    r.sa2_code;
    
CREATE OR REPLACE VIEW polls_agg AS
SELECT 
    r.sa2_code,
    count(q.polling_place_id) as "total_polls"
FROM 
    regions r JOIN polls q ON ST_CONTAINS(r.geometry, ST_TRANSFORM(q.geometry, 7844))
GROUP BY
    r.sa2_code;
    
SELECT *
FROM catchment_agg JOIN stops_agg USING (sa2_code) JOIN polls_agg USING (sa2_code)
LIMIT 10;
""" 

query(conn, sql)

,sa2_code,total_schools,schools_per_1000_young,total_stops,total_polls


#### Calculations

We now combine all aggregate calculations in one view.

In [ ]:
sql = """
CREATE OR REPLACE VIEW all_aggs as
SELECT 
    *
FROM
    retail_agg FULL OUTER JOIN health_agg ON (sa2_code)
               FULL OUTER JOIN catchment_agg ON (sa2_code)
               FULL OUTER JOIN stops_agg ON (sa2_code)
               FULL OUTER JOIN polls_agg ON (sa2_code)
               
SELECT *
FROM all_aggs
LIMIT 10;
"""

Finding mean and standard deviation for all components of well-resourced score in order to later find Z scores. THIS IS WHERE WE WOULD OUTLIER DETECT ETC.

In [38]:
sql = """
CREATE OR REPLACE VIEW metrics as
SELECT 
    avg(retail_per_1000) as "mean_retail",
    sd(retail_per_1000) as "sd_retail",
    avg(health_per_1000) as "mean_health",
    sd(health_per_1000) as "sd_health",
    avg(total_stops) as "mean_stops",
    sd(total_stops) as "sd_stops",
    avg(total_polls) as "mean_polls",
    sd(total_polls) as "sd_polls",
    avg(schools_per_1000_young) as "mean_schools",
    avg(schools_per_1000_young) as  "sd_schools"
FROM
    all_aggs
    
SELECT *
FROM metrics
LIMIT 10;
"""

query(conn, sql)

Finally we calculate our z scores, and overall well-resourced score for each sa2 code.

In [39]:
sql = """
CREATE OR REPLACE VIEW scores as
SELECT 
    sa2_code,
    (retail_per_1000 - mean_retail) / sd_retail as "retail_z",
    (health_per_1000 - mean_health) / sd_health as "health_z",
    (total_stops - mean_stops) / sd_stops as "stops_z",
    (total_polls - mean_polls) / sd_polls as "polls_z",
    (schools_per_1000_young - mean_schools) / sd_schools as "schools_z",
    [(retail_per_1000 - mean_retail) / sd_retail] + 
        [(health_per_1000 - mean_health) / sd_health] + 
        [(total_stops - mean_stops) / sd_stops] +
        [(total_polls - mean_polls) / sd_polls] }
        [(schools_per_1000_young - mean_schools)] as "score"
FROM
    all_aggs CROSS JOIN metrics;
    
SELECT *
FROM scores
LIMIT 10;
"""

query(conn, sql)

#### OLD MASSIVE JOIN

In [ ]:
# this isnt ideal as it will duplicate each row for each catchment, then this set of rows for each stop, then again for each poll
'''
sql="""
CREATE OR REPLACE VIEW joined AS
SELECT *
FROM regions r LEFT JOIN business b USING (sa2_code)
               LEFT JOIN population p USING (sa2_code)
               JOIN catchments c ON ST_INTERSECTS(r.geometry, c.geometry)
               JOIN stops s ON ST_CONTAINS(r.geometry, s.geometry)
               JOIN polls q ON ST_CONTAINS(r.geometry, q.geometry)
GROUP BY ;
               
SELECT *
FROM joined
LIMIT 10;
""" 

query(conn, sql)
'''

### Task 3: Extending the "well-resourced" Score & Visualisations 

#### Income dataset

In [40]:
income = pd.read_csv("Income.csv")
print(income.shape)
print(income.columns)


# Filtering for only the entries with income data in greater Sydney
income = income[(income["earners"]!="np") & (income["median_age"]!="np")
        & (income["median_income"]!="np") & (income["mean_income"]!="np")]
income = income.loc[income["sa2_code"].isin(regions["sa2_code"])]

# Correcting datatypes
income = income.astype({"earners":"int64", "median_age":"int64", "median_income":"int64", "mean_income":"int64"})
print(income.dtypes)

(576, 6)
Index(['sa2_code', 'sa2_name', 'earners', 'median_age', 'median_income',
       'mean_income'],
      dtype='object')
sa2_code          int64
sa2_name         object
earners           int64
median_age        int64
median_income     int64
mean_income       int64
dtype: object


In [41]:
# Removing unnecessary columns
income.drop("sa2_name", axis=1, inplace=True)

# Adding to postgresql database
sql = """
DROP TABLE IF EXISTS income;
CREATE TABLE income (
    sa2_code INTEGER PRIMARY KEY REFERENCES regions (sa2_code),
    earners INTEGER NOT NULL,
    median_age INTEGER NOT NULL,
    median_income INTEGER NOT NULL,
    mean_income INTEGER NOT NULL
);"""
conn.execute(sql);

income.to_sql("income", conn, schema="sa2", if_exists="append", index=False)


query(conn, "SELECT * FROM income LIMIT 5;")

,sa2_code,earners,median_age,median_income,mean_income
0,102011028,4788,47,52450,71977
1,102011029,6647,49,48724,64621
2,102011030,2919,49,46228,61545
3,102011031,7962,48,48292,67162
4,102011032,11439,42,51999,62478


#### Points/Features of Interest dataset
https://portal.spatial.nsw.gov.au/server/rest/services/NSW_Features_of_Interest_Category/FeatureServer

In addition, since GitHub can only store files of less than 100MB the zip file will need to be extracted here:

In [42]:
! unzip -o Points_Of_Interest

Archive:  Points_Of_Interest.zip
  inflating: Points_Of_Interest_EPSG4326.json  


In [43]:
import json
features_json = json.loads(open("Points_Of_Interest_EPSG4326.json").read())

In [44]:
features = pd.DataFrame(features_json["Points_Of_Interest"]["features"])
features.head()

,type,geometry,properties
0,Feature,"{'type': 'Point', 'coordinates': [152.12202209...","{'topoid': 500000000, 'poigroup': 9, 'poitype'..."
1,Feature,"{'type': 'Point', 'coordinates': [152.29868781...","{'topoid': 500005504, 'poigroup': 3, 'poitype'..."
2,Feature,"{'type': 'Point', 'coordinates': [152.33785614...","{'topoid': 500005505, 'poigroup': 3, 'poitype'..."
3,Feature,"{'type': 'Point', 'coordinates': [152.34181334...","{'topoid': 500005507, 'poigroup': 3, 'poitype'..."
4,Feature,"{'type': 'Point', 'coordinates': [152.47882472...","{'topoid': 500012781, 'poigroup': 3, 'poitype'..."


As we can see, we will need to expand out the geometry and properties columns. Since all of the locations are provided as points:

In [ ]:
# Expanding out the two columns
features_geometry = features["geometry"].apply(pd.Series)
features_properties = features["properties"].apply(pd.Series)

In [ ]:
# Combining the two datasets together
features_df = pd.concat([features_geometry[["coordinates"]], features_properties], axis=1)
features_df.head()

Since for our analysis we are only interested in the number of recreational spaces (such as parks, camping grounds and picnic areas), we will filter the dataset for only the rows that have a poigroup value of 3:

In [ ]:
# Selecting only the named recreational places of interest
features_df = features_df[features_df["poigroup"] == 3]
features_df = features_df[features_df["poilabeltype"] == "NAMED"]

# Inspecting 
print(features_df.shape)
print(features_df.dtypes)
features_df.head()

In [ ]:
# Dropping unnecessary columns
features_df = features_df[["coordinates", "topoid", "poitype", "poiname"]]
features_df.head()

In [ ]:
# Converting DataFrame to GeoDataFrame
features_gdf = gpd.GeoDataFrame(features_df, geometry=gpd.points_from_xy(features_df.coordinates.str[0], features_df.coordinates.str[1])).set_crs(epsg=4283)
features_gdf.drop("coordinates", axis=1, inplace=True)

# Renaming columns
features_gdf.rename({"topoid":"location_id", "poitype":"location_type", "poiname":"location_name"}, axis=1, inplace=True)

features_gdf.head()

In [ ]:
# Adding to postgresql database
sql = """
DROP TABLE IF EXISTS features;
CREATE TABLE features (
    location_id INTEGER PRIMARY KEY,
    location_type VARCHAR(50) NOT NULL,
    location_name VARCHAR(100) NOT NULL,
    geometry GEOMETRY(POINT, 4283) NOT NULL
);"""
conn.execute(sql);

features_gdf.to_postgis("features", conn, schema="sa2", if_exists="append", index=False)

query(conn, "SELECT * FROM features LIMIT 5;")

#### Car Crashes Dataset
Data from https://opendata.transport.nsw.gov.au/dataset/nsw-crash-data

In [ ]:
#reading in the data
crashes = pd.read_excel("crashes.xlsx")
print(crashes.shape)
print(crashes.columns)
print(crashes.dtypes)

Convert DataFrame into GeoDataFrame with geopandas:

In [ ]:
#convert into gdf
crashes_gdf = gpd.GeoDataFrame(crashes, geometry = gpd.points_from_xy(crashes.Longitude, crashes.Latitude)).set_crs(epsg=4326)

#remove unnecessary columns and rows
crashes_gdf.columns = [c.lower().replace(' ', '').replace('-', '').replace('.', '') for c in crashes_gdf.columns]
crashes_gdf.drop(crashes_gdf[crashes_gdf.yearofcrash != 2021].index, inplace=True)
crashes_gdf.drop(columns=['latitude', 'longitude', 'streetofcrash', 'streettype', 'twohourintervals', 'distance', 'direction', 'identifyingfeature', 'identifyingfeaturetype', 'town', 'routeno', 'schoolzonelocation', 'schoolzoneactive', 'typeoflocation', 'urbanisation', 'conurbation1', 'alignment', 'primarypermanentfeature', 'primarytemporaryfeature', 'primaryhazardousfeature', 'streetlighting', 'roadsurface', 'surfacecondition', 'weather', 'naturallighting', 'signalsoperation', 'othertrafficcontrol', 'speedlimit', 'roadclassification(admin)', 'rumcode', 'rumdescription', 'dcacode', 'dcadescription', 'dcasupplement', 'firstimpacttype', 'keytutype', 'othertutype'], inplace=True)
crashes_gdf.head()

Now add into postgresql database

In [ ]:
sql = """
DROP TABLE IF EXISTS crashes;
CREATE TABLE crashes (
    crashid INTEGER PRIMARY KEY,
    degreeofcrash VARCHAR(100) NOT NULL,
    degreeofcrashdetailed VARCHAR(100) NOT NULL,
    reportingyear INTEGER NOT NULL,
    yearofcrash INTEGER NOT NULL,
    monthofcrash VARCHAR(100) NOT NULL,
    dayofweekofcrash VARCHAR(100) NOT NULL,
    lga VARCHAR(100) NOT NULL,
    nooftrafficunitsinvolved INTEGER NOT NULL,
    nokilled INTEGER NOT NULL,
    noseriouslyinjured INTEGER NOT NULL,
    nomoderatelyinjured INTEGER NOT NULL,
    nominorotherinjured INTEGER NOT NULL,
    geometry GEOMETRY(POINT, 4326) NOT NULL
);
"""

conn.execute(sql)

crashes_gdf.to_postgis('crashes', conn, schema="sa2", if_exists="append", index=False)

query(conn, "SELECT * FROM crashes LIMIT 5;")

### Task 4:  Additional Analysis

## Extra commands

In [ ]:
# Checks that all tables are in sa2 schema
inspect(db).get_table_names(schema="sa2")

In [ ]:
# Close connection to database
#pgdisconnect(db, conn)